# DCBO + 'Bayesian Optimisation over Multiple Continuous and Categorical Inputs'

In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd
import sys
import matplotlib.pyplot as plt
from networkx import nx_agraph
import pygraphviz
from copy import deepcopy
# sys.path.append("../src/")
sys.path.append("..")

In [3]:
from src.test_functions.synfunc_1d import obj_func_bo
from src.mab_cons.OnehotEnconding import OnehotEncoding

To optimize 2d synthetic function (1 categorical + 1 continuous) with the number of arms C=6 (fixed) and the batch size=[1, 5, 10].

To test:
- Bayesian Optimization for Categorical and Category-Specific Continuous Inputs *(need to make ordinal kernel)*
- Dealing with Categorical and Integer-valued Variables in Bayesian Optimization with Gaussian Processes


In [48]:
import numpy as np

# Transforming discrete values according to the method by Merchan and Lobato.


def twogaussian(x, shifter=0.0):
    y = (
        np.exp(-((x - (shifter * 0.1) - 2) ** 2))
        + np.exp(-((x - (shifter * 0.1) - 6) ** 2) / 10)
        + 1 / ((x + (shifter * 0.1)) ** 2 + 1)
    )

    return y.reshape(-1, 1) + shifter  # Min=0.2, Max=1.4


# objective function for BO-based methods
def obj_func_bo(arm, x):
    # maximize function
    auc = twogaussian(x, shifter=arm * 0.5)

    return auc


f = obj_func_bo

c_bound_dim = 6
x_bound_dim = 1
x_bound = [-2.0, 10.0]

trials = 10  # 10
n_arm = c_bound_dim
budget = 40  # 40
batch_list = [1]  # [1, 5, 10]
n_init = 2 # Number of initial points [per dimension]

bounds_categorical_c = [
    {"name": "x{}".format(d + 1), "type": "continuous", "domain": (0, 1)} for d in range(c_bound_dim)
]
bounds_categorical_x = [
    {"name": "x{}".format(d + c_bound_dim + 1), "type": "continuous", "domain": (x_bound[0], x_bound[1])}
    for d in range(x_bound_dim)
]
bounds = list(np.append(bounds_categorical_c, bounds_categorical_x))

# Some conversions that happen inside the MAB_Cons class in Nguyen's implementation
n_dim = len(bounds)

n_init_point = n_arm * n_init
Xinit_ker = np.zeros((n_init_point, n_dim))
for c in range(n_arm):
    # map to one-hot encoding
    x1 = np.zeros(n_arm)
    x1[c] = 1
    for i in range(n_init):
        x2 = np.array(
            [np.random.uniform(bounds[d]["domain"][0], bounds[d]["domain"][1], 1)[0] for d in range(n_arm, n_dim)]
        )
        x = np.append(x1, x2)
        Xinit_ker[i + c * n_init, :] = x

Yinit_ker = []
for i in range(n_init_point):
    x = Xinit_ker[i, :]
    # map one-hot encoding of categorical variable to arm index
    arm = np.argmax(x[:n_arm])
    y = f(arm, x[n_arm:])
    Yinit_ker.append(y)

In [49]:
bounds

[{'name': 'x1', 'type': 'continuous', 'domain': (0, 1)},
 {'name': 'x2', 'type': 'continuous', 'domain': (0, 1)},
 {'name': 'x3', 'type': 'continuous', 'domain': (0, 1)},
 {'name': 'x4', 'type': 'continuous', 'domain': (0, 1)},
 {'name': 'x5', 'type': 'continuous', 'domain': (0, 1)},
 {'name': 'x6', 'type': 'continuous', 'domain': (0, 1)},
 {'name': 'x7', 'type': 'continuous', 'domain': (-2.0, 10.0)}]

In [50]:
Xinit_ker.round(2).shape

(12, 7)

In [51]:
Xinit_ker.round(2)

array([[ 1.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  2.34],
       [ 1.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  6.61],
       [ 0.  ,  1.  ,  0.  ,  0.  ,  0.  ,  0.  ,  5.77],
       [ 0.  ,  1.  ,  0.  ,  0.  ,  0.  ,  0.  ,  5.95],
       [ 0.  ,  0.  ,  1.  ,  0.  ,  0.  ,  0.  ,  4.72],
       [ 0.  ,  0.  ,  1.  ,  0.  ,  0.  ,  0.  ,  6.71],
       [ 0.  ,  0.  ,  0.  ,  1.  ,  0.  ,  0.  ,  9.29],
       [ 0.  ,  0.  ,  0.  ,  1.  ,  0.  ,  0.  ,  9.77],
       [ 0.  ,  0.  ,  0.  ,  0.  ,  1.  ,  0.  ,  6.05],
       [ 0.  ,  0.  ,  0.  ,  0.  ,  1.  ,  0.  ,  5.88],
       [ 0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  1.  , -0.22],
       [ 0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  1.  ,  0.08]])

In [52]:
# np.argmax(x[:n_arm])
x[n_arm:]

array([0.08276774])

## Transform input in RBF kernal space

### Categorical

In [53]:
X = Xinit_ker
X.round(2)

array([[ 1.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  2.34],
       [ 1.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  6.61],
       [ 0.  ,  1.  ,  0.  ,  0.  ,  0.  ,  0.  ,  5.77],
       [ 0.  ,  1.  ,  0.  ,  0.  ,  0.  ,  0.  ,  5.95],
       [ 0.  ,  0.  ,  1.  ,  0.  ,  0.  ,  0.  ,  4.72],
       [ 0.  ,  0.  ,  1.  ,  0.  ,  0.  ,  0.  ,  6.71],
       [ 0.  ,  0.  ,  0.  ,  1.  ,  0.  ,  0.  ,  9.29],
       [ 0.  ,  0.  ,  0.  ,  1.  ,  0.  ,  0.  ,  9.77],
       [ 0.  ,  0.  ,  0.  ,  0.  ,  1.  ,  0.  ,  6.05],
       [ 0.  ,  0.  ,  0.  ,  0.  ,  1.  ,  0.  ,  5.88],
       [ 0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  1.  , -0.22],
       [ 0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  1.  ,  0.08]])

In [55]:
def transform_to_one_hot(vector):
    one_hot = np.zeros(len(vector))
    active_val = np.argmax(vector)
    one_hot[active_val] = 1
    return one_hot

categorical_dim = n_arm # Total number of categories in our categorical variable

np.array([np.append(transform_to_one_hot(x_val[:categorical_dim]),x_val[categorical_dim:]) for x_val in X]).round(2)

array([[ 1.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  2.34],
       [ 1.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  6.61],
       [ 0.  ,  1.  ,  0.  ,  0.  ,  0.  ,  0.  ,  5.77],
       [ 0.  ,  1.  ,  0.  ,  0.  ,  0.  ,  0.  ,  5.95],
       [ 0.  ,  0.  ,  1.  ,  0.  ,  0.  ,  0.  ,  4.72],
       [ 0.  ,  0.  ,  1.  ,  0.  ,  0.  ,  0.  ,  6.71],
       [ 0.  ,  0.  ,  0.  ,  1.  ,  0.  ,  0.  ,  9.29],
       [ 0.  ,  0.  ,  0.  ,  1.  ,  0.  ,  0.  ,  9.77],
       [ 0.  ,  0.  ,  0.  ,  0.  ,  1.  ,  0.  ,  6.05],
       [ 0.  ,  0.  ,  0.  ,  0.  ,  1.  ,  0.  ,  5.88],
       [ 0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  1.  , -0.22],
       [ 0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  1.  ,  0.08]])

### Ordinal

In [58]:
XX = np.hstack((np.random.randint(0,10,12).reshape(-1,1),X[:,-1].reshape(-1,1)))
XX

array([[ 0.        ,  2.3403384 ],
       [ 7.        ,  6.60926197],
       [ 4.        ,  5.77067366],
       [ 0.        ,  5.9450693 ],
       [ 2.        ,  4.71616478],
       [ 7.        ,  6.71229473],
       [ 9.        ,  9.28917361],
       [ 4.        ,  9.76813244],
       [ 6.        ,  6.05357254],
       [ 0.        ,  5.88385259],
       [ 1.        , -0.22055158],
       [ 7.        ,  0.08276774]])

In [60]:
discrete_dim = 0
np.array([np.append(np.around(x_val[:discrete_dim], 0), x_val[discrete_dim:]) for x_val in XX])

array([[ 0.        ,  2.3403384 ],
       [ 7.        ,  6.60926197],
       [ 4.        ,  5.77067366],
       [ 0.        ,  5.9450693 ],
       [ 2.        ,  4.71616478],
       [ 7.        ,  6.71229473],
       [ 9.        ,  9.28917361],
       [ 4.        ,  9.76813244],
       [ 6.        ,  6.05357254],
       [ 0.        ,  5.88385259],
       [ 1.        , -0.22055158],
       [ 7.        ,  0.08276774]])

In [62]:
Xsq = np.sum(np.square(X),1)
r2 = -2.*np.dot(X) + (Xsq[:,None] + Xsq[None,:])
util.diag.view(r2)[:,]= 0. # force diagnoal to be zero: sometime numerically a little negative
r2 = np.clip(r2, 0, np.inf)

TypeError: dot() missing 1 required positional argument: 'b'